In [13]:
from google.cloud import storage
import os
import pandas as pd

In [15]:
pd.__version__

'0.23.4'

In [2]:
project=os.environ['PROJECT_ID']

In [6]:
client = storage.Client(project=project)

In [7]:
for b in client.list_buckets():
    print(b.name)

artifacts.ml-productive-pipeline-53122.appspot.com
ml-productive-pipeline-53122
ml-productive-pipeline-53122_cloudbuild


In [8]:
from google.cloud import bigquery
client = bigquery.Client(project=project)

In [22]:
query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100 * SUM(IF(answer_count > 0, 1, 0)) / COUNT(*), 1) AS Percent_Questions_with_Answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2016
ORDER BY
  Year;
        """
#df = client.query(query).to_dataframe()
#df.head()
query_job = client.query(query)
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

AttributeError: no row field 'name'

In [21]:
df

TypeError: __init__() got an unexpected keyword argument 'max_rows'

   Year  Number_of_Questions  Percent_Questions_with_Answers
0  2009               343139                            99.7
1  2010               693332                            99.1
2  2011              1198587                            97.2
3  2012              1642687                            94.6
4  2013              2056613                            91.6
5  2014              2160361                            88.5
6  2015              2214389                            86.4